In [1]:
from pathlib import Path

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim

## Loading the data

### Forecasts

We want to test for a linear relationship between the 0-hour predictions for longwave and shortwave flux (`DLWRF_SFC` and `DSWRF_SFC`) and the target data collected at the solar farm. The solar farm is located on the map at position (91, 81) in the (y, x) dimensions.

We can use the `uga_solar.data.nam` package to load the forecasts. Loading forecasts can take a while, so you should only load a small subset while working. The function `nam.open_range(start, end)` allows us to open open a subset of the data by timestamp.

The data is returned as an xarray `Dataset`. We can use the xarray API to select specific variables and slice any dimension. The `.sel` method allows us to select along dimensions by their coordinate labels, and `.isel` allows us to select by position index. We use these to slice the y, x, and forecast dimensions.

In [2]:
from uga_solar.data import nam
inputs = nam.open_range('2016-12-01T00:00', '2017-11-01T00:00')
inputs = inputs[['DSWRF_SFC', 'DLWRF_SFC']]
inputs = inputs.isel(y=slice(90, 93))
inputs = inputs.isel(x=slice(80, 83))
inputs = inputs.isel(forecast=0)

### Labels

The targets can be loaded with the `uga_solar.data.ga_power` module. The `open_aggregate` function opens all target logs of the given module and computes their average over a 1-hour time interval. The result is cached to disk. The value returned is a pandas `Dataframe`.

Module 7 has most of the important stuff. We select column 5 as the target, which is the ventilated pyranometer on the 2-axis array. We also drop rows with NaNs.

In [3]:
from uga_solar.data import ga_power
targets = ga_power.open_aggregate(7)
targets = targets[5]
targets = targets.dropna()

### PyTorch DataLoader

Now that we have our data, we need to adapt it to the PyTorch `DataSet` interface which allows it to be iterated over in batches with a PyTorch `DataLoader`. The `uga_solar.data` package has some tools to help us out.

The `XarrayDataset` class (name subject to change) adapts our data to the PyTorch interface. It takes an xarray Dataset as its first argument followed by any number of pandas DataFrames. The DataFrames are joined against the `reftime` dimension of the xarray dataset.

The `CrossValLoader` class is a factory for PyTorch `DataLoader`s with cross-validation support. For each fold, it generates 3 dataloaders for the train set, test set, and validation set. If we have $n$ samples divided into $k$ folds, then the test and validation sets each contain $n/k$ samples while the train set contains the remaining $n(k-2)/k$ samples.

We create a loader as an example and use it to load a single batch. We can use the inputs and targets returned from this batch to help define the shape of our network.

In [4]:
from uga_solar.data import XarrayDataset, CrossValLoader
dataset = XarrayDataset(inputs, targets)
cv = CrossValLoader(dataset)
epoch = iter(cv.load_val(0))
batch = next(epoch)
x, y = batch

print('inputs:', x.shape, x.type())
print('targets:', y.shape, y.type())

inputs: torch.Size([32, 2, 1, 3, 3]) torch.DoubleTensor
targets: torch.Size([32]) torch.DoubleTensor


## Modeling

### The Network

The `Net` class defines our network. We're just using a simple linear regression for now.

In [5]:
class Net(nn.Module):
    def __init__(self, in_shape, out_shape):
        super().__init__()
        self.in_size = int(np.prod(in_shape))
        self.out_size = int(np.prod(out_shape))
        self.in_shape = in_shape
        self.out_shape = out_shape
        self.linear = nn.Linear(self.in_size, self.out_size)
        
    def forward(self, x):
        x = x.view(-1, self.in_size)
        x = self.linear(x)
        x = x.view(-1, *self.out_shape)
        return x

The `Estimator` class wraps a network, cost function, and optimizer into a scikit-learn-like interface.

The network and cost functions are both instances of [`torch.nn.Module`](http://pytorch.org/docs/master/nn.html) and the optimizer is an instance of [`torch.optim.Optimizer`](http://pytorch.org/docs/master/optim.html).

In [6]:
class Estimator:
    def __init__(self, net, cost, optimizer, dtype=None):
        if dtype is None:
            if torch.cuda.is_available():
                dtype = torch.cuda.FloatTensor
            else:
                dtype = torch.FloatTensor
        self.dtype = dtype
        self.net = net.type(self.dtype)
        self.cost = cost
        self.optimizer = optimizer
        
    def partial_fit(self, x, y):
        self.net.train(True)
        x = autograd.Variable(x).type(self.dtype)
        y = autograd.Variable(y).type(self.dtype)
        self.optimizer.zero_grad()
        h = self.net(x)
        j = self.cost(h, y)
        j.backward()
        self.optimizer.step()
        return j
        
    def predict(self, x):
        self.net.train(False)
        x = autograd.Variable(x).type(self.dtype)
        h = self.net(x)
        return h
    
    def score(self, x, y, criteria=None):
        self.net.train(False)
        criteria = criteria or self.cost
        x = autograd.Variable(x).type(self.dtype)
        y = autograd.Variable(y).type(self.dtype)
        h = self.net(x)
        j = criteria(h, y)
        return j

### Running an experiment

Now we have everything we need to run an experiment.

For each fold, we create a new estimator. We train it for some number of epochs, and at the end of each epoch, we evaluate it against the validation set. Finally, we evaluate each model on their respective test sets.

Currently we're only testing the final model from each fold. A more complete experiment would instead save the model which had the best validation loss across all epochs, and test that model instead.

In [ ]:
cv = CrossValLoader(dataset, n_folds=5)

for i in range(5):
    print('>>> fold', i)
    net = Net(x.shape[1:], y.shape[1:])
    cost = nn.MSELoss()
    optimizer = optim.Adam(net.parameters())
    estimator = Estimator(net, cost, optimizer)
    train_set = cv.load_train(i)
    val_set = cv.load_val(i)
    test_set = cv.load_test(i)
    
    for epoch in range(50):
        print(f'epoch {epoch+1:02d} ', end='')
        
        # train
        loss = 0
        n = len(train_set)
        for x, y in train_set:
            j = estimator.partial_fit(x, y)
            loss += j * (len(x) / n)
            print(end='.', flush=True)
        loss = loss.data.cpu().numpy().squeeze()
        print(f'[Train loss: {float(loss):8.2e}]', end='')
            
        # evaluate - val set
        loss = 0
        n = len(val_set)
        for x, y in val_set:
            j = estimator.score(x, y)
            loss += j * (len(x) / n)
        loss = loss.data.cpu().numpy().squeeze()
        print(f'[Val loss: {float(loss):8.2e}]')
        
    # evaluate - test set
    loss = 0
    n = len(test_set)
    for x, y in test_set:
        j = estimator.score(x, y)
        loss += j * (len(x) / n)
    loss = loss.data.cpu().numpy().squeeze()
    print(f'[Test loss: {float(loss):8.2e}]')

>>> fold 0
epoch 01 .....................[Train loss: 1.03e+07][Val loss: 1.15e+04]
epoch 02 .....................[Train loss: 6.29e+06][Val loss: 4.88e+04]
epoch 03 .....................[Train loss: 4.43e+06][Val loss: 2.04e+05]
epoch 04 .....................[Train loss: 3.79e+06][Val loss: 3.54e+05]
epoch 05 .....................[Train loss: 3.64e+06][Val loss: 4.56e+05]
epoch 06 .....................[Train loss: 3.64e+06][Val loss: 5.14e+05]
epoch 07 .....................[Train loss: 3.65e+06][Val loss: 5.43e+05]
epoch 08 .....................[Train loss: 3.64e+06][Val loss: 5.56e+05]
epoch 09 .....................[Train loss: 3.63e+06][Val loss: 5.62e+05]
epoch 10 .....................[Train loss: 3.61e+06][Val loss: 5.63e+05]
epoch 11 .........[Val loss: 5.64e+05]
epoch 13 .....................[Train loss: 3.53e+06][Val loss: 5.65e+05]
epoch 14 .....................[Train loss: 3.51e+06][Val loss: 5.66e+05]
epoch 15 .....................[Train loss: 3.48e+06][Val loss: 5.68e+05]
e